In [1]:
import csv
import re

In [2]:
!pip install requests


In [2]:
import requests

# Define the public URL of GRch 38 S3 file
public_url = 'https://rp2clinicalgenomicsdavidmandia.s3.amazonaws.com/output_GCF_000001405.40-RS_2023_03_knownrefseq_alns.sam'
local_file_name = 'GRCh38.sam'

# Download the file
response = requests.get(public_url)

# Check if the request was successful
if response.status_code == 200:
    with open(local_file_name, 'wb') as f:
        f.write(response.content)
    print(f"File downloaded successfully and saved as {local_file_name}")
else:
    print(f"Failed to download file. Status code: {response.status_code}")


File downloaded successfully and saved as GRCh38.sam


In [3]:
# Define the public URL of your GRCh37 S3 file
public_url_2 = 'https://rp2clinicalgenomicsdavidmandia.s3.amazonaws.com/outputGCF_000001405.25_GRCh37.p13_knownrefseq_alns.sam'
local_file_name_2 = 'GRCh37.sam'

# Download the file
response_2 = requests.get(public_url_2)

# Check if the request was successful
if response_2.status_code == 200:
    with open(local_file_name_2, 'wb') as f:
        f.write(response_2.content)
    print(f"File downloaded successfully and saved as {local_file_name_2}")
else:
    print(f"Failed to download file. Status code: {response_2.status_code}")


File downloaded successfully and saved as GRCh37.sam


In [4]:
def parse_sam(file_path):
    alignments = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('@'):
                # Skip header lines
                continue
            #print(line)
            fields = line.strip().split('\t')
            read_id = fields[0]
            genome_ref = fields[2]
            cigar = fields[5]
            alignments.append((read_id, genome_ref, cigar))
    return alignments

In [5]:
# def analyze_cigar(cigar):
#     operations = re.findall(r'(\d+)([MIDNSHP=X])', cigar)
#     analysis = {}
#     for length, op in operations:
#         if op not in analysis:
#             analysis[op] = 0
#         analysis[op] += int(length)
#     return analysis

In [6]:
# Path to your SAM file
sam_file_path = 'GRCh37.sam'

# Parse the SAM file
alignments_37 = parse_sam(sam_file_path)

In [7]:
# Path to your SAM file
sam_file_path = 'GRCh38.sam'

# Parse the SAM file
alignments_38 = parse_sam(sam_file_path)

In [8]:
alignments_37[0]

('NR_046018.2', 'NC_000001.10', '354=385N109=499N1189=')

In [9]:
alignments_38[0]

('NR_046018.1', 'NC_000001.11', '354=385N41=1X67=499N106=1X1081=')

In [10]:
def identify_short_indels(cigar_string):
    # Regular expression to extract length and operation from CIGAR string
    cigar_pattern = re.compile(r'(\d+)([MIDNSHP=X])')
    
    # List to store short indels
    short_indels = []
    
    # Iterate through each operation in the CIGAR string
    for length, op in re.findall(cigar_pattern, cigar_string):
        # Check if the operation is an insertion (I) or deletion (D)
        if op == 'I' or op == 'D':
            # Convert length to integer
            length = int(length)
            
            # Check if the length of the indel is less than or equal to 3
            if length <= 3:
                # Append the operation and length to the short indels list
                short_indels.append((op, length))
    
    return short_indels

# # Example usage:
# cigar = '10M2D5I20M'
# short_indels = identify_short_indels(cigar)
# print("Short Indels:", short_indels)


In [11]:
def return_indels(GrCh_list):
    indels = {}
    for ref_cigar in GrCh_list:
        short_indels = identify_short_indels(ref_cigar[2])
        if len(short_indels)> 0:
            indels[ref_cigar[0]] = short_indels
        else: 
            continue
    return indels
    
        

In [12]:
hg38_indels = return_indels(alignments_38)

In [13]:
hg38_indels

{'NR_039983.1': [('D', 1)],
 'NM_024796.1': [('I', 1)],
 'NM_005101.2': [('D', 1)],
 'NM_005101.1': [('D', 1)],
 'NM_198576.1': [('I', 2)],
 'NM_003327.1': [('I', 2), ('D', 1), ('I', 1), ('D', 1), ('D', 1), ('I', 3)],
 'NM_016176.1': [('I', 1), ('D', 1), ('D', 1)],
 'NM_016547.1': [('I', 1), ('D', 1), ('D', 1)],
 'NM_080605.1': [('D', 2), ('D', 1)],
 'NM_058167.1': [('D', 1)],
 'NM_032179.1': [('I', 1)],
 'NM_001039577.1': [('D', 1)],
 'NM_030937.3': [('D', 1)],
 'NM_030937.2': [('D', 2),
  ('I', 1),
  ('I', 2),
  ('I', 1),
  ('I', 2),
  ('I', 1),
  ('D', 1),
  ('D', 1)],
 'NM_014188.1': [('D', 2)],
 'NM_001787.1': [('D', 1)],
 'NM_033486.1': [('D', 1)],
 'NM_033487.1': [('D', 1)],
 'NM_033488.1': [('D', 1)],
 'NM_033489.1': [('D', 1)],
 'NM_033490.1': [('D', 1)],
 'NM_033492.1': [('D', 1)],
 'NM_033493.1': [('D', 1)],
 'NM_001110781.1': [('D', 1)],
 'NM_033527.1': [('D', 1)],
 'NM_033528.1': [('D', 1)],
 'NM_033529.1': [('D', 1)],
 'NM_033531.1': [('D', 1)],
 'NM_033532.1': [('D', 1)]

In [14]:
!git lfs install
!git lfs track "*.sam"
!git add .gitattributes
!

Updated Git hooks.
Git LFS initialized.
Tracking "*.sam"
